
* * * *
### 1. Introduction

This dataset is a collection of images that are commonly used to train machine learning and computer vision algorithms. It is one of the most widely used datasets for machine learning research. The CIFAR-10 dataset contains 60,000 32x32 color images in 10 different classes. The 10 different classes represent airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks. There are 6,000 images of each class. In this competition, your goal is to correctly identify different object from a dataset of tens of thousands of color images.
* * * *
This notebook is divided in the following sections:
1. Introduction
1. Data preparation
    1. Load data
    1. Normalization, Reshape and Label encoding
    1. Visualize test and train sample
1. Model Building
    1. Split training and valdiation set
    1. Define the model architechture
    1. Set the optimizer and annealer
    1. Data augmentation
    1. Train model
1. Evaluate the model
    1. Training and validation curves
    1. Visualize Prediction
1. Prediction and submition
    1. Predict and Submit results

In [22]:
# Importing relevant packages and modules
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from  os import listdir
# import backend
import tensorflow as tf
# Model architecture
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D
from keras.layers import MaxPool2D, Activation, MaxPooling2D
from keras.layers.normalization import BatchNormalization

from keras.datasets import cifar10
# Annealer
from keras.callbacks import LearningRateScheduler

# Data processing
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.preprocessing import image

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns


# Progressor
from tqdm import tqdm
import h5py

Using TensorFlow backend.

Bad key "text.kerning_factor" on line 4 in
C:\Users\Arnav\Anaconda3\envs\gpu\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [1]:
# Load the CIFAR10 Datset using keras
# Usage Example:
#(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

### 2. Data Prepration
**Load Data**

In [2]:
#Print the shape for test and train data

In [3]:
#Check dimensions of the loaded data

In [4]:
# Distribution of classes in training and testing samples

#Create a bar graph having ddistribution of all the classses to check for any imbalanced classes

*Next, we must do the following:*
**Normalization, Reshape and Label Encoding**

***Normalization***
* Perform a grayscale normalization to reduce the effect of illumination's differences.
* If we perform normalization, CNN works faster.
***Reshape***
* Train and test images (32 x 32)
* Reshape all data to -1*28x28x1 4D matrices.

***Label Encoding***
* Encode labels to one hot vectors
* Example:
> 2 => [0,0,1,0,0,0,0,0,0,0] <br>
> 4 => [0,0,0,0,1,0,0,0,0,0]

In [5]:
## Function to reshape and scaling image
# Create a function to reshape the images to (-1,32,32,3)  

In [6]:
# Training and Testing data processing
# Call the function to reshape, scale images on training and testing data

# Label processing
# One hot encode the training labels

In [7]:
# Label encoding
from sklearn.preprocessing import LabelEncoder
# Use sklearn LabelEncoder to encode test labels

**Visualize test and train sample**
##### Train sample

In [9]:
# visualizing training samples
# Plot the first 40 training images on a 10 x 4 grid 
# This will help us to visualize the train data

##### Test sample

In [10]:
# visualizing test samples
# Plot the first 40 test images on a 10 x 4 grid 
# This will help us to visualize the test data

### 3. Model Building: 

**Split training and valdiation set**

In [11]:
# split training and validation set.
# Use train_test_split to split the traing data into train and validation test 

## Define the model architechture

****

**CNNs and their layers:**

**Convolutional neural networks (CNNs)** are the current **state-of-the-art model architecture** for image classification tasks. CNNs apply a series of filters to the raw pixel data of an image to extract and learn higher-level features, which the model can then use for classification. CNNs contains different components:

***Convolutional layers***, which apply a specified number of convolution filters to the image. For each subregion, the layer performs a set of mathematical operations to produce a single value in the output **feature map**. Convolutional layers then typically apply a **ReLU activation** function) to the output to introduce **nonlinearities** into the model.

***BatchNormalization layers***, Batch Normalization is a technical trick to make training faster.

***Dropout layers***, Dropout is a regularization method, where the layer randomly replaces a proportion of its weights to zero for each training sample. This forces the net to learn features in a distributed way, not relying to much on a particular weight, and therefore improves generalization.

***Activation layers***, Activation functions are really important for a any Neural Network to learn and make sense of something really complicated and Non-linear complex functional mappings between the inputs and response variable.They introduce **non-linear properties** to our Network.

***Pooling layers***, which downsample the image data extracted by the convolutional layers to reduce the dimensionality of the feature map in order to decrease processing time. A commonly used pooling algorithm is max pooling, which extracts subregions of the feature map (e.g., 2x2-pixel tiles), keeps their maximum value, and discards all other values. it gives **rotation invariant** feature extraction ability to model. 

***Dense (fully connected) layers***, which perform classification on the features extracted by the convolutional layers and downsampled by the pooling layers. In a dense layer, every node in the layer is connected to every node in the preceding layer.

Typically, a CNN is composed of a stack of convolutional modules that perform feature extraction. Each module consists of a convolutional layer followed by a pooling layer. The last convolutional module is followed by one or more dense layers that perform classification. The final dense layer in a CNN contains a single node for each target class in the model (all the possible classes the model may predict), with a softmax activation function to generate a value between 0–1 for each node (the sum of all these softmax values is equal to 1). We can interpret the softmax values for a given image as relative measurements of how likely it is that the image falls into each target class.



In [12]:
# BUILD CONVOLUTIONAL NEURAL NETWORKS
# Build a CNN model
# Compile this model
# Print the model summary

**Set the optimizer and annealer**

We train once with a smaller learning rate to ensure convergence. We then speed things up, only to reduce the learning rate by 10% every epoch. Keras has a function for this:

In [14]:
import math
# learning rate schedule
# Create a function to change learning rate for every epoch (This step is optional but ensures performance)

# learning schedule callback
# Use Keras LearningRateScheduler

**Data augmentation**

In order to avoid overfitting problem, we need to expand artificially our dataset. We can make existing dataset even larger by altering the training data with small transformations.

For the data augmentation, we can:

* Randomly rotate some training images by 10 degrees
* Randomly Zoom by 10% some training images
* Randomly shift images horizontally by 10% of the width
* Randomly shift images vertically by 10% of the height
* Randomly flip images horizontally.

Not: We cannot apply a vertical_flip since it could misclassify symetrical object like aeroplane and bird, truck and ships.

In [13]:
# Data augumetation
# Perform Data augmentation using ImageDataGenerator
# Data generator model to train and validation set
# Set batch size

**Generated sample**

In [14]:
# visualizing augumented image
# Print 10 augmented variants of an image in a row
# Do this for 3 images

**Train model**

In [17]:
# traing parameters
# Set epoch and Batch size

In [15]:
# Fit the model with the required parametes

In [16]:
# Print the loss and accuracy of validation data for the fit

### 4. Evaluate the model

**Training and validation curves**



In [17]:
# Plot the loss and accuracy curves for training and validation 

**Visualize Result**

In [18]:
# making predictions
# PREVIEW PREDICTIONS
# Create a plot to visualize the results

### 5. Prediction and submition
**Predict and Submit results**

In [19]:
# prediction on test
# Now use the test data and save the predictions of test data in a list

In [20]:
# Convert the predicted numbers into labels

In [21]:
# Save these predicted labels in a csv file

### Conclusion: 

We can now classify images using CNNs which are a very powerful and handy when it comes to neural networks. Additionally, you may run this on other models/ classifiers and compare your results with CNNs. Ussually, Convolutional Neural Networks will outperform them.